In [ ]:
!pip install keras4torch
!pip install einops


import os
import keras4torch
from   keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import torch
import torch.nn    as nn
import torch.optim as optim
import  torch.nn.functional as     F
import numpy       as np
import pandas      as pd
from copy import deepcopy
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import STab
from STab import MyClassLoss,CatMap,Num_Cat
from   STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

In [ ]:
##Load DAta
X_test=pd.DataFrame(np.load('Data/helena/N_test.npy')).astype(np.float64)
y_test=pd.DataFrame(np.load('Data/helena/y_test.npy')).astype(np.int32)[0]

X_train=pd.DataFrame(np.load('Data/helena/N_train.npy')).astype(np.float64)
Y_train=pd.DataFrame(np.load('Data/helena/y_train.npy')).astype(np.int32)[0]


X_valid=pd.DataFrame(np.load('Data/helena/N_val.npy')).astype(np.float64)
y_valid=pd.DataFrame(np.load('Data/helena/y_val.npy')).astype(np.int32)[0]

In [ ]:
#Normalise In
scalerX = QuantileTransformer(output_distribution='normal',
                              n_quantiles=max(min(X_train.shape[0]//30,1000),10),
                              subsample=int(1e9),)


scalerX.fit(X_train)

X_train = scalerX.transform(X_train).astype(np.float64)
X_valid = scalerX.transform(X_valid).astype(np.float64)
X_test  = scalerX.transform(X_test).astype(np.float64)

In [ ]:
chpfilename='saved_vanilla/savefileHE'

In [ ]:
def True_ACC(p,t):
                return np.mean(np.where(np.round(p)==np.round(t),1,0))

In [ ]:
Or_model = MainModel(
    categories        = (),
    num_continuous    = 27,
    dim               = 96  ,
    dim_out           = 101,
    depth             = 7,
    heads             = 8,
    attn_dropout      = 0.25 ,
    ff_dropout        = 0.25,
    U                 = 2,
    cases             = 16

)
no_model = Num_Cat(Or_model,num_number=27,classes=101)
model    = keras4torch.Model(no_model,).build([27])

#Warm Up Train
no_model.reset_Sample_size(1)
optimizer=torch.optim.AdamW(model.parameters(),lr=0.0001,weight_decay=0.0001,)
sch=torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.00001, total_iters=5,  verbose=False)
model.compile(optimizer=optimizer, loss=MyClassLoss(0.1,1), metrics=['accuracy'])
callbacks=[LRScheduler(sch)]
model.fit(X_train, Y_train.values,
        epochs=5, batch_size=512,
        validation_data=(X_valid,y_valid.values),
        verbose=2,validation_batch_size=1024,
        callbacks=callbacks)

#Main Train
no_model.reset_Sample_size(64)
optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.000001,)
model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
scheduler =LRScheduler( torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5,min_lr=0.00001))
callbacks=[scheduler,ModelCheckpoint(chpfilename,monitor='val_acc',mode='max')]
model.fit(X_train, Y_train.values,
        epochs=100, batch_size=512,
        validation_data=(X_valid,y_valid.values),

        verbose=2,validation_batch_size=256,
        callbacks=callbacks)


model.load_weights(chpfilename)

no_model.reset_Sample_size(1)
logits=0

for i in range(0,64):

        logits+=pd.DataFrame(model.predict(X_test,batch_size=4096))


Test = (True_ACC(logits.idxmax(axis=1).values.reshape((-1)),y_test.values.reshape((-1))))

print('Test Score:',Test)



[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert int32 to int64, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert int32 to int64, this could lead to extra memory usage.
Train on 41724 samples, validate on 10432 samples:
Epoch 1/5 - 90s - loss: 4.3719 - acc: 0.0083 - val_loss: 4.3657 - val_acc: 0.0083 - lr: 1e-09
Epoch 2/5 - 84s - loss: 4.1542 - acc: 0.0471 - val_loss: 4.0325 - val_acc: 0.0696 - lr: 2e-05
Epoch 3/5 - 84s - loss: 3.8622 - acc: 0.1021 - val_loss: 3.8042 - val_acc: 0.1117 - lr: 4e-05
Epoch 4/5 - 84s - loss: 3.6608 - acc: 0.1409 - val_loss: 3.6195 - val_acc: 0.1473 - lr: 6e-05
Epoch 5/5 - 82s - loss: 3.4787 - acc: 0.1775 - val_loss: 3.4428 - val_acc: 0.1851 - lr: 8e-05
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert int32 to int64, this could lead to extra memory usage.